# Report on people with single open request for pads or diapers.

In [2]:
import pandas as pd

from bam_core.lib.airtable import Airtable

airtable = Airtable()



In [3]:
# get all assistance requests records
raw_records = airtable.assistance_requests.all()


In [4]:
# create a dataframe
df = pd.DataFrame([r.get('fields') for r in raw_records])

In [6]:
from collections import Counter

# get all phone numbers w/ one open request
single_req_pns = {k for k, v in Counter(df['Phone Number']).items() if v == 1}

# pull open pad/diaper request views
pad_view = airtable.assistance_requests.all(view='P2W - Open Pads Requests')
diaper_view = airtable.assistance_requests.all(view='P2W - Open Baby Diaper Requests')

pad_df = pd.DataFrame([r.get('fields') for r in pad_view])
diaper_df = pd.DataFrame([r.get('fields') for r in diaper_view])

# pull all requests from pad/diaper request views with phone number from single reqs list
def get_single_req_pns(df):
    phones = set(df['Phone Number'])
    return phones & single_req_pns

pad_single_req_pns = get_single_req_pns(pad_df)
diaper_single_req_pns = get_single_req_pns(diaper_df)

In [20]:
# write all single requests to csv
df[df['Phone Number'].isin(pad_single_req_pns - diaper_single_req_pns)].to_csv('pad_single_reqs.csv')
df[df['Phone Number'].isin(diaper_single_req_pns - pad_single_req_pns)].to_csv('diaper_single_reqs.csv')
df[df['Phone Number'].isin(pad_single_req_pns & diaper_single_req_pns)].to_csv('pad_and_diaper_single_reqs.csv')

492
164
